In [1]:
#import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
#read in merged and cleaned data
df = pd.read_csv('../../data/merged_cleaned_data/cases_merged_data_svi_val_cols.csv')
df

,Unnamed: 0,FIPS,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,...,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,first_year_cases
0,0,1001,594.443459,55200,23315,21115,8422,1065,29372,4204,...,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,6589
1,1,1009,644.830460,57645,24222,20600,8220,909,22656,7861,...,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,6444
2,2,1013,776.838201,20025,10026,6708,4640,567,20430,2141,...,19011.0,78.563680,76.109761,76.623924,69.058104,79.956648,67.920284,72.773953,41.789629,2097
3,3,1015,605.867251,115098,53682,45033,20819,4628,24706,12620,...,22231.0,79.439032,79.955121,77.918741,54.063568,76.745724,67.456150,68.292794,29.845243,14224
4,4,1017,596.560643,33826,16981,13516,5531,773,22827,4383,...,21532.0,76.995358,78.156771,75.891100,67.343775,79.128558,66.397785,69.554441,41.644857,3488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3053,3053,48131,1793.476183,11355,5592,3511,2751,482,17864,2386,...,19853.0,79.125428,78.895880,76.629575,60.576045,73.670302,64.571017,68.007770,50.959826,1214
3054,3054,48505,998.411980,14369,6388,4405,5609,621,17228,3226,...,16007.0,79.355639,79.572483,74.378252,77.443239,76.386871,74.001471,73.609838,47.134744,1760
3055,3055,48507,1297.406535,12131,4344,3509,4150,421,13350,2719,...,13393.0,78.392216,76.024682,75.848196,76.967659,77.303576,70.010162,71.121990,65.403060,1844
3056,3056,48127,1328.884075,10663,4408,3309,3148,694,18121,2137,...,19528.0,79.457964,79.141556,66.677924,73.140121,71.659882,68.065499,66.378831,61.756683,1707


In [3]:
#set index to FIPS
df = df.set_index(df['FIPS'])
df= df.drop(columns = ['FIPS','Unnamed: 0'])

In [4]:
#create variable for case% for each counties population
df['case_pct'] = df['first_year_cases']/df['E_TOTPOP']*100
df['case_pct'].head()

FIPS
1001    11.936594
1009    11.178767
1013    10.471910
1015    12.358164
1017    10.311595
Name: case_pct, dtype: float64

In [5]:
df['case_pct'].describe()

count    3058.000000
mean        9.426600
std         3.045809
min         0.000000
25%         7.713422
50%         9.466675
75%        11.176131
max        38.010657
Name: case_pct, dtype: float64

In [6]:
# bin and cut the case_pct column into 2 classifications
q = df['case_pct'].quantile(.5)
bins = [0, q , 40]
labels = ['low','high']
df['case_class'] = pd.cut(df['case_pct'], bins, labels = labels)
df['case_class']

FIPS
1001     high
1009     high
1013     high
1015     high
1017     high
         ... 
48131    high
48505    high
48507    high
48127    high
48247    high
Name: case_class, Length: 3058, dtype: category
Categories (2, object): ['low' < 'high']

In [7]:
df['case_class'].value_counts()

high    1529
low     1505
Name: case_class, dtype: int64

In [8]:
# impact_cols = ['Empathy',
#  'EPL_GROUPQ',
#  'Agreeableness',
#  'Employment Rate',
#  'EPL_UNEMP',
#  'Openness',
#  'E_AGE65',
#  'EP_AGE17',
#  'Conscientiousness',
#  'SPL_THEME4',
#  'Income Per Capita',
#  'E_POV',
#  'E_MINRTY',
#  'Conflict Awareness',
#  'Extraversion',
#  'EP_SNGPNT',
#  'F_MOBILE',
#  'Work Ethic',
#  'E_AGE17',
#  'EP_PCI',
#  'Risk Taking',
#  'E_UNEMP',
#  'AREA_SQMI',
#  'E_NOHSDP',
#  'EPL_PCI',
#  'E_HU',
#  'E_NOVEH',
#  'E_LIMENG',
#  'EPL_SNGPNT',
#  'E_UNINSUR',
#  'EPL_AGE17',
#  'Belief In Science',
#  'EPL_LIMENG',
#  'EPL_NOVEH',
#  'Entrepreneurship',
#  'SPL_THEME3',
#  'E_HH',
#  'EP_MUNIT',
#  'EPL_MOBILE',
#  'Collectivism',
#  'RPL_THEME2',
#  'E_MUNIT',
#  'EPL_POV',
#  'EP_UNINSUR',
#  'dem_pct',
#  'F_SNGPNT',
#  'EPL_MUNIT',
#  'EP_NOHSDP',
#  'EPL_DISABL',
#  'Hopefulness',
#  'E_MOBILE']

In [9]:
# impact_cols_reduced = ['Empathy',
#  'EPL_GROUPQ',
#  'Agreeableness',
#  'Employment Rate',
#  'EPL_UNEMP',
#  'Openness',
#  'E_AGE65',
#  'EP_AGE17',
#  'Conscientiousness',
#  'SPL_THEME4',
#  'Income Per Capita',
#  'E_POV',
#  'first_year_cases',
#  'E_MINRTY',
#  'Conflict Awareness',
#  'Extraversion',
#  'EP_SNGPNT',
#  'F_MOBILE',
#  'Work Ethic',
#  'E_AGE17',
#  'EP_PCI',
#  'Risk Taking',
#  'E_UNEMP',
#  'AREA_SQMI',
#  'E_NOHSDP',
#  'EPL_PCI',
#  'E_HU',
#  'E_NOVEH',
#  'E_LIMENG',
#  'EPL_SNGPNT',
#  'E_UNINSUR',
#  'EPL_AGE17']

In [10]:
#drop unneeded columns
df = df.drop('case_pct', axis =1)


In [11]:
#turn case % classifications into binary 
df = pd.get_dummies(df, columns = ['case_class'])
df

,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,E_AGE17,...,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,first_year_cases,case_class_low,case_class_high
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,594.443459,55200,23315,21115,8422,1065,29372,4204,8050,13369,...,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,6589,0,1
1009,644.830460,57645,24222,20600,8220,909,22656,7861,10233,13468,...,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,6444,0,1
1013,776.838201,20025,10026,6708,4640,567,20430,2141,3806,4566,...,76.109761,76.623924,69.058104,79.956648,67.920284,72.773953,41.789629,2097,0,1
1015,605.867251,115098,53682,45033,20819,4628,24706,12620,19386,25196,...,79.955121,77.918741,54.063568,76.745724,67.456150,68.292794,29.845243,14224,0,1
1017,596.560643,33826,16981,13516,5531,773,22827,4383,6409,7006,...,78.156771,75.891100,67.343775,79.128558,66.397785,69.554441,41.644857,3488,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,1793.476183,11355,5592,3511,2751,482,17864,2386,2025,2962,...,78.895880,76.629575,60.576045,73.670302,64.571017,68.007770,50.959826,1214,0,1
48505,998.411980,14369,6388,4405,5609,621,17228,3226,1999,4835,...,79.572483,74.378252,77.443239,76.386871,74.001471,73.609838,47.134744,1760,0,1
48507,1297.406535,12131,4344,3509,4150,421,13350,2719,1665,3583,...,76.024682,75.848196,76.967659,77.303576,70.010162,71.121990,65.403060,1844,0,1


In [12]:
df['case_class_low'].value_counts()

0    1553
1    1505
Name: case_class_low, dtype: int64

In [13]:
#seperate targets and features
## should i drop the number of cases?
X = df.drop(columns = ['case_class_high','case_class_low','first_year_cases']).values
y=df['case_class_low'].values

In [14]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
len(X_train[0])

41

In [17]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 80
# hidden_nodes_layer1 = 100
# hidden_nodes_layer2 = 80
# hidden_nodes_layer3 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4200      
                                                                 
 dense_1 (Dense)             (None, 80)                8080      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 
Total params: 12,361
Trainable params: 12,361
Non-trainable params: 0
_________________________________________________________________


2022-08-12 19:53:53.292872: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=150)

Epoch 1/150
72/72 [==============================] - 2s 2ms/step - loss: 1540.1071 - accuracy: 0.5338
Epoch 2/150
72/72 [==============================] - 0s 2ms/step - loss: 1356.1594 - accuracy: 0.5464
Epoch 3/150
72/72 [==============================] - 0s 2ms/step - loss: 649.5179 - accuracy: 0.5678
Epoch 4/150
72/72 [==============================] - 0s 3ms/step - loss: 967.5751 - accuracy: 0.5377
Epoch 5/150
72/72 [==============================] - 0s 3ms/step - loss: 617.1084 - accuracy: 0.5635
Epoch 6/150
72/72 [==============================] - 0s 2ms/step - loss: 257.5381 - accuracy: 0.5905
Epoch 7/150
72/72 [==============================] - 0s 2ms/step - loss: 885.9995 - accuracy: 0.5530
Epoch 8/150
72/72 [==============================] - 0s 2ms/step - loss: 702.4294 - accuracy: 0.5874
Epoch 9/150
72/72 [==============================] - 0s 2ms/step - loss: 284.5565 - accuracy: 0.6014
Epoch 10/150
72/72 [==============================] - 0s 2ms/step - loss: 292.6890 - accu

72/72 [==============================] - 0s 2ms/step - loss: 13.7364 - accuracy: 0.6293
Epoch 82/150
72/72 [==============================] - 0s 2ms/step - loss: 9.7380 - accuracy: 0.6350
Epoch 83/150
72/72 [==============================] - 0s 2ms/step - loss: 10.6139 - accuracy: 0.6324
Epoch 84/150
72/72 [==============================] - 0s 2ms/step - loss: 18.5519 - accuracy: 0.5909
Epoch 85/150
72/72 [==============================] - 0s 2ms/step - loss: 19.0751 - accuracy: 0.6293
Epoch 86/150
72/72 [==============================] - 0s 2ms/step - loss: 11.2522 - accuracy: 0.6088
Epoch 87/150
72/72 [==============================] - 0s 2ms/step - loss: 8.6576 - accuracy: 0.6524
Epoch 88/150
72/72 [==============================] - 0s 2ms/step - loss: 11.4022 - accuracy: 0.6127
Epoch 89/150
72/72 [==============================] - 0s 2ms/step - loss: 10.9611 - accuracy: 0.6280
Epoch 90/150
72/72 [==============================] - 0s 2ms/step - loss: 14.4058 - accuracy: 0.5901
Epoch

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

24/24 - 0s - loss: 1.2091 - accuracy: 0.5712 - 192ms/epoch - 8ms/step
Loss: 1.209130883216858, Accuracy: 0.5712418556213379


In [21]:
y_pred=nn.predict(X_test)
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred)
con_mat


24/24 [==============================] - 0s 1ms/step


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[375,   3],
       [385,   2]], dtype=int32)>

In [22]:
np.count_nonzero(y_test==0)

378

In [23]:
# nn.save('saved_models/aug_10_reduced_features_model.h5')

In [24]:
# nn_imported = tf.keras.models.load_model('saved_models/aug_8_reduced_features_model.h5')

In [25]:
# # Evaluate the completed model using the test data
# model_loss, model_accuracy = nn_imported.evaluate(X_test,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [26]:
# y_pred=nn_imported.predict(X_test)
# con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred)
# con_mat

In [27]:
# pred = pd.DataFrame(y_pred)
# pred

In [28]:
# pred.loc[pred[0]==1]